<a href="https://colab.research.google.com/github/trevor-johnson/w266_reddit_summarization/blob/master/W266_Reddit_Project_AndrewWorking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Data and Create Subset

In [ ]:
import tensorflow_datasets as tfds 
data = tfds.load(name='reddit')

In [ ]:
import tensorflow as tf
import pandas as pd
from pprint import pprint

In [ ]:
data_train = data['train']

In [ ]:
data_train

<PrefetchDataset element_spec={'author': TensorSpec(shape=(), dtype=tf.string, name=None), 'body': TensorSpec(shape=(), dtype=tf.string, name=None), 'content': TensorSpec(shape=(), dtype=tf.string, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'normalizedBody': TensorSpec(shape=(), dtype=tf.string, name=None), 'subreddit': TensorSpec(shape=(), dtype=tf.string, name=None), 'subreddit_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'summary': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
#Get a subset of data (1000 rows)
ds_subset = data_train.take(1000)
ds_subset

<TakeDataset element_spec={'author': TensorSpec(shape=(), dtype=tf.string, name=None), 'body': TensorSpec(shape=(), dtype=tf.string, name=None), 'content': TensorSpec(shape=(), dtype=tf.string, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'normalizedBody': TensorSpec(shape=(), dtype=tf.string, name=None), 'subreddit': TensorSpec(shape=(), dtype=tf.string, name=None), 'subreddit_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'summary': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
#Convert to a pandas dataframe
df = pd.DataFrame(ds_subset)
#Get first row of data
first_row_of_body = df.iloc[0]['body']
first_row_of_summary = df.iloc[0]['summary']
first_row_of_summary

<tf.Tensor: shape=(), dtype=string, numpy=b'I think I may have emotionally cheated, should I tell my gf?'>

In [ ]:
#Convert the tf constant to a bytes, converts byte to str
my_str = tf.constant(first_row_of_body)
my_str_npy = my_str.numpy()
my_str_npy
article_to_summarize = my_str_npy.decode()
article_to_summarize

"I  was talking to this girl I've met today, and I'm not so sure what happened. It was nice seeing her after almost 3 weeks last talking with her, and she seems like a cool person to hang out with. I realized however, that I want to try to be good friends with her. We walked around and talked a while, and before I knew what happened I asked and she gave me her digits. \n\nWe proceeded to text each other back and forth about lots of bullshit, and I suddenly asked myself; is this emotionally cheating? To be honest, I do feel as if I did. \n\nI understand I (19M)am in a monogamous relationship with my gf(17), and I want communication between us to be crystal clear. I feel guilty, and an obligation to tell her of what I did. Should I tell my gf about this, or stop myself in the future before something like this happens again? I am emotionally confused right now, and could use some advice.\n\nTL;DR I think I may have emotionally cheated, should I tell my gf?"

In [ ]:
!pip install -q transformers

In [ ]:
!pip install -q sentencepiece

In [ ]:
!pip install -q datasets

# T5 for Summarization

In [ ]:
#T5 Model
from transformers import T5Tokenizer, TFT5ForConditionalGeneration

T5_model = TFT5ForConditionalGeneration.from_pretrained("google/t5-v1_1-base")
tokenizer = T5Tokenizer.from_pretrained("google/t5-v1_1-base")


All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at google/t5-v1_1-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
T5_model.summary()

Model: "tft5_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 shared (TFSharedEmbeddings)  multiple                 24674304  
                                                                 
 encoder (TFT5MainLayer)     multiple                  84954240  
                                                                 
 decoder (TFT5MainLayer)     multiple                  113275008 
                                                                 
 lm_head (Dense)             multiple                  24674304  
                                                                 
Total params: 247,577,856
Trainable params: 247,577,856
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Tell T5 model to do the summarization task
PROMPT = 'summarize: '
T5_article_to_summarize = PROMPT + article_to_summarize



In [ ]:
#Inputs for T5 Model
inputs = tokenizer(T5_article_to_summarize, max_length=1024, truncation=True, return_tensors="tf")
inputs

{'input_ids': <tf.Tensor: shape=(1, 244), dtype=int32, numpy=
array([[21603,    10,    27,    47,  2508,    12,    48,  3202,    27,
           31,   162,  1736,   469,     6,    11,    27,    31,    51,
           59,    78,   417,   125,  2817,     5,    94,    47,  1245,
         2492,   160,   227,   966,   220,  1274,   336,  2508,    28,
          160,     6,    11,   255,  1330,   114,     3,     9,  1633,
          568,    12,  5168,    91,    28,     5,    27,  5723,   983,
            6,    24,    27,   241,    12,   653,    12,    36,   207,
          803,    28,   160,     5,   101,     3,  6412,   300,    11,
         6812,     3,     9,   298,     6,    11,   274,    27,  2124,
          125,  2817,    27,  1380,    11,   255,  1891,   140,   160,
            3,  9206,     7,     5,   101,  8669,    15,    26,    12,
         1499,   284,   119,   223,    11,  7444,    81,  1995,    13,
         8434,     7, 10536,     6,    11,    27,  8247,  1380,  1512,
          117, 

In [ ]:
# Generate Summary
summary_ids = T5_model.generate(inputs["input_ids"] 
)
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

", and I'm not sure what I did. I'm not sure what I"


In [ ]:
# Generate Summary
summary_ids = T5_model.generate(inputs["input_ids"], 
                              num_beams=3,
                              no_repeat_ngram_size=1,
                              min_length=15,
                              max_length=20)
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

", and I'm not sure what happened. It was a good girl to see"


# Pegasus For Summarization

In [ ]:
from transformers import PegasusTokenizer, TFPegasusForConditionalGeneration

pega_model = TFPegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")

All model checkpoint layers were used when initializing TFPegasusForConditionalGeneration.

All the layers of TFPegasusForConditionalGeneration were initialized from the model checkpoint at google/pegasus-xsum.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFPegasusForConditionalGeneration for predictions without further training.


In [ ]:
pega_model.summary()

Model: "tf_pegasus_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFPegasusMainLayer)  multiple                  569748480 
                                                                 
Total params: 569,844,583
Trainable params: 569,748,480
Non-trainable params: 96,103
_________________________________________________________________


In [ ]:
inputs = tokenizer(article_to_summarize, max_length=1024, truncation=True, return_tensors="tf")
inputs

{'input_ids': <tf.Tensor: shape=(1, 225), dtype=int32, numpy=
array([[  125,   140,  1767,   112,   136,  2092,   125,   131,   261,
         1676,   380,   108,   111,   125,   131,   208,   146,   167,
          334,   180,  2032,   107,   168,   140,   948,  1749,   215,
          244,   744,   296,   899,   289,  1767,   122,   215,   108,
          111,   265,  1045,   172,   114,  1274,   465,   112,  4330,
          165,   122,   107,   125,  3577,   802,   108,   120,   125,
          245,   112,   508,   112,   129,   234,   594,   122,   215,
          107,   184,  3998,   279,   111,  4337,   114,   277,   108,
          111,   269,   125,  1606,   180,  2032,   125,  1049,   111,
          265,  1422,   213,   215, 15865,   107,   184, 17925,   112,
         1352,   276,   176,   247,   111,  5381,   160,  1425,   113,
        29143, 16201,   108,   111,   125,  5008,  1049,  1150,   206,
          117,   136, 10071, 17092,   152,   413,   129,  3200,   108,
          125, 

In [ ]:
# Generate Summary
summary_ids = pega_model.generate(inputs["input_ids"])
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)

"I'm a 17-year-old girl."


In [ ]:
# Generate Summary
summary_ids = pega_model.generate(inputs["input_ids"], 
                              num_beams=4,
                              no_repeat_ngram_size=2,
                              min_length=5,
                              max_length=25)
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0], compact=True)



"I'm a 17-year-old girl, and I have a question."


# BART for Summarization

In [ ]:
#Generate Bart Model
from transformers import BartTokenizer, TFBartForConditionalGeneration

bart_model = TFBartForConditionalGeneration.from_pretrained("facebook/bart-large")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large")

All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


In [ ]:
bart_model.summary()

Model: "tf_bart_for_conditional_generation_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model (TFBartMainLayer)     multiple                  406291456 
                                                                 
Total params: 406,341,721
Trainable params: 406,291,456
Non-trainable params: 50,265
_________________________________________________________________


In [ ]:
inputs = tokenizer([article_to_summarize], max_length=1024, truncation=True, return_tensors="tf")
inputs

{'input_ids': <tf.Tensor: shape=(1, 232), dtype=int32, numpy=
array([[    0,   100,  1437,    21,  1686,     7,    42,  1816,    38,
          348,  1145,   452,     6,     8,    38,   437,    45,    98,
          686,    99,  1102,     4,    85,    21,  2579,  1782,    69,
           71,   818,   155,   688,    94,  1686,    19,    69,     6,
            8,    79,  1302,   101,    10,  3035,   621,     7,  6713,
           66,    19,     4,    38,  5426,   959,     6,    14,    38,
          236,     7,   860,     7,    28,   205,   964,    19,    69,
            4,   166,  3203,   198,     8,  3244,    10,   150,     6,
            8,   137,    38,  1467,    99,  1102,    38,   553,     8,
           79,   851,   162,    69, 15769,     4,  1437, 50118, 50118,
          170, 18254,     7,  2788,   349,    97,   124,     8,  7264,
           59,  3739,     9, 37568,     6,     8,    38,  6017,   553,
         2185,   131,    16,    42, 13582, 14153,   116,   598,    28,
         5322, 

In [ ]:
# Generate Summary
summary_ids = bart_model.generate(inputs["input_ids"], 
                              num_beams=3,
                              no_repeat_ngram_size=2,
                              min_length=10,
                              max_length=100)
pprint(tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False), compact=True)
yhat = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


KeyboardInterrupt: ignored

In [ ]:
#article_to_summarize
yhat = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
pprint(yhat, compact=True)

# Rogue Score

In [ ]:
!pip install rouge-score

In [ ]:
# package from google research: https://github.com/google-research/google-research/tree/master/rouge
from rouge_score import rouge_scorer

In [ ]:
print("true:")
pprint(first_row_of_summary)

print("\n\nprediction:")
pprint(yhat)

print("\n\nMetrics:")
# set up type of scores you want
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rougeL'], use_stemmer=True)
scores = scorer.score(target=article_to_summarize, prediction=yhat)
scores

#Loop through 10 articles and avg rouge score (BART model)


In [ ]:
scores_list = []
for i in range(10):

  #Get first row of data
  first_row_of_body = df.iloc[i]['body']
  first_row_of_summary = df.iloc[i]['summary']

  #Convert the tf constant to a bytes, converts byte to str
  my_str = tf.constant(first_row_of_body)
  my_str_npy = my_str.numpy()
  article_to_summarize = my_str_npy.decode()

  #Create Summary Model
  summary_ids = bart_model.generate(inputs["input_ids"], 
                                num_beams=3,
                                no_repeat_ngram_size=2,
                                min_length=10,
                                max_length=100)
  yhat = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
  # Rogue Scores
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rouge3', 'rougeL'], use_stemmer=True)
  scores = scorer.score(target=article_to_summarize, prediction=yhat)
  print(i)
  scores_list.append(scores)

In [ ]:
scores_list

# Average Rouge Scores Across Examples

In [ ]:
#Combine keys in list
Precision_vals = []
Recall_vals = []
Fmeasure_vals = []
for i in range(10):
  #Precision
  Precision_vals.append(scores_list[i]['rouge1'][0])
  #Recall
  Recall_vals.append(scores_list[0]['rouge1'][1])
  #Fmeasure
  Fmeasure_vals.append(scores_list[0]['rouge1'][2])

Avg_Precision = sum(Precision_vals)/len(Precision_vals)
Avg_Recall = sum(Recall_vals)/len(Recall_vals)
Avg_Fmeasure = sum(Fmeasure_vals)/len(Fmeasure_vals)

print("Average Precision is " + str(Avg_Precision))
print("Average Recall is " + str(Avg_Recall))
print("Average Fmeasure is " + str(Avg_Fmeasure))

